# Использование дополнительных стилей CSS и кода JS

Приложения Dash отображаются в веб-браузере с помощью CSS и JavaScript. При загрузке страницы Dash предоставляет небольшой HTML-шаблон, содержащий ссылки на CSS и JavaScript, необходимые для рендеринга приложения.

Базовый вариант добавления CSS или JS: создать папку с именем `assets` в корневом каталоге приложения и положить в нее свои файлы CSS и JavaScript. Dash будет автоматически обрабатывать все файлы, включенные в эту папку. По умолчанию URL-адрес для запроса ресурсов будет /assets, но вы можете настроить его с помощью аргумента `assets_url_path` для `dash.Dash`.

Автоматически включаются:
* CSS файлы с расширением .css (включаются после стилей компонентов Dash)
* JS файлы с расширением .js
* Иконка сайта favicon.ico

Файлы включаются в _алфавитном порядке_.

Дополнительно можно:
- прикладывать файлы (изображения) в каталог assets
- прикладывать файлы из CDN
- добавлять дополнительные файлы с JS и CSS из внешних источников: `Dash.external_scripts`, `Dash.external_stylesheets`.
- переопределять используемую версию plotly.js: `dcc.Graph` использует plotly.js для визуализации, вместе с dash идет своя версия, но при желании ее можно заменить
- подключить Tex: `external_scripts=["https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.5/MathJax.js?config=TeX-MML-AM_CHTML" ]`


In [13]:
%%file 05_extra_resources/app.py
import plotly.express as px
from dash import Dash, dcc, Input, Output, html
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/gapminderDataFiveYear.csv')
app = Dash(
    __name__, 
    # external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css'],
    assets_folder="assets"
)
app.layout = html.Div(
    [
        dcc.Graph(id='graph-with-slider'),
        dcc.Slider(
            df['year'].min(),
            df['year'].max(),
            step=None,
            value=df['year'].min(),
            marks={str(year): str(year) for year in df['year'].unique()},
            id='year-slider'
        ),
        html.H2('Горизонтальная ось:'),
        dcc.RadioItems(
            ['Linear', 'Log'],
            value='Log',
            id='xaxis-type'
        )
    ]
)

# callback может принимать несколько входных параметров
@app.callback(
    # модифицируем фигуру на основе
    Output('graph-with-slider', 'figure'),
    # значения слайдера и селектора
    Input('year-slider', 'value'),
    Input('xaxis-type', 'value')
)
def _(selected_year, xaxis_type):
    # функция обратного вызова не должна модифицировать глобальные переменные
    filtered_df = df[df.year == selected_year]
    fig = px.scatter(
        filtered_df, 
        x="gdpPercap", 
        y="lifeExp",
        size="pop", 
        color="continent", 
        hover_name="country",
        log_x=(xaxis_type=='Log'), 
        size_max=55
    )
    return fig


if __name__ == "__main__":
    app.run_server(debug=True)

Overwriting 05_extra_resources/app.py
